# Прогнозирование оттока клиентов банка

### Постановка бизнес-задачи, описание предметной области.  

#### Предметная область
Рассматривается задача прогнозирования оттока клиентов банка. Отток (churn) - прекращение использования клиентом банковских услуг. Удержание клиентов - приоритет для бизнеса из-за высокой стоимости их привлечения.  

---
#### Контекст бизнеса

**Банки стремятся**:
- Выявлять клиентов с высоким риском ухода
- Повышать уровень удержания
- Персонализировать предложения

**Ключевые аспекты:**
- Своевременное выявление оттока
- Оптимизация маркетинговых и удерживающих кампаний
- Увеличение прибыли за счёт лояльных клиентов

---
#### Бизнес-задача
Разработать модель машинного обучения для предсказания ухода клиента на основе его характеристик.

**Цель:**
- Снизить отток клиентов
- Поддержки стабильной прибыли банков
- Улучшения стратегий взаимодействия с клиентами

---
#### Описание набора данных
**Источник:** Kaggle - [Bank Customer Churn Prediction](https://www.kaggle.com/datasets/shubhammeshram579/bank-customer-churn-prediction)  
**Объём:** ~10 000 клиентов  
**Целевая переменная:** `Exited` (1 — клиент ушёл, 0 — остался)

**Основные признаки**

| Признак                | Тип            | Описание                                                  |
|------------------------|----------------|-----------------------------------------------------------|
| `CustomerId`           | Integer        | Уникальный идентификатор клиента                          |
| `Surname`              | String         | Фамилия клиента                                           |
| `CreditScore`          | Integer        | Кредитный рейтинг клиента                                 |
| `Geography`            | String         | Страна проживания                                         |
| `Gender`               | String         | Пол клиента                                               |
| `Age`                  | Integer        | Возраст клиента                                           |
| `Tenure`               | Integer        | Лет сотрудничества                                        |
| `Balance`              | Float          | Остаток на счёте                                          |
| `NumOfProducts`        | Integer        | Кол-во продуктов банка                                    |
| `HasCrCard`            | Integer        | Наличие кредитной карты                                   |
| `IsActiveMember`       | Float          | Активность клиента                                        |
| `EstimatedSalary`      | Float          | Предполагаемый доход                                      |
| `Exited`               | Integer        | Ушел ли клиент или нет (целевая переменная)               |


---

### Предобработка данных

In [ ]:
import pandas as pd

#### Знакомство с данными

In [ ]:
# Загрузка дата сета
data_set = pd.read_csv("../data/churn.csv")
data_set

In [ ]:
# первые 10 строк
data_set.head(10)

In [ ]:
# последние 10 строк
data_set.tail(10)

In [ ]:
# случайные 10 строк
data_set.sample(10)

In [ ]:
# название колонок в датасете
data_set.columns

In [ ]:
# размер датасета
data_set.index

In [ ]:
# Общие сведения о датасете 
data_set.info()

#### Проблемы при беглом анализе
При беглом осмотре можно сразу заметить несколько проблем в данном датасете:  
1. Колонка RowNumber - по сути является счетчиком строк, что не нужно, т.к порядок строки в данных не влияет на результат
2. Колонка CustomerId - уникальный идентификатор клиента, по сути не влияет на результат
3. В колонке Age значения представлены в виде чисел с плавающей точкой (float), хотя возраст исчисляется целочисленно
4. Колонки HasCrCard и IsActiveMember, возможно, содержат числа с плавающей точкой. В описании данных четко указано, что значениями могут быть только 1 (Да) и 0 (Нет)
5. Колонка Gender, возможно, имеет только 2 значения - Male и Female, можно ввести ассоциацию: 1 (Male) и 2 (Female)
6. Колонка Geography имеет только 3 значения - можно ввести ассоциацию: 1 (France), 2 (Spain), 3 (Germany)
7. Были замечены пустые значения
8. Названия колонок не соответствуют "змеиному регистру"
9. У некоторых имен имеются спец символы
10. У некоторых фамилий обнаружены спец. символы (например H? в строке с индексом 9)

#### Детальный анализ датасета и каждого столбца

In [ ]:
# типы данных каждого столбца
data_set.dtypes

In [ ]:
# точное измерение памяти
data_set.memory_usage(deep=True)

In [ ]:
# поиск пустых значений по столбцам
data_set.isnull().sum()

In [ ]:
# детальный анализ каждого столбца

for column in data_set.columns:
    print(f"Колонка {column}")
    print(f"Тип данных: {data_set[column].dtype}")
    print(f"Количество пустых значений: {data_set[column].isnull().sum()}")
    print(f"Количество уникальных значений: {data_set[column].nunique()}")
    print(f"Уникальные значения: {data_set[column].unique()}")
    print(f"{data_set[column].describe()}")
    print()

При детальном анализе выявлены пустые значения, а также обнаружены столбцы, где можно привести данные в нормальную форму, например в колонке Age нужно привести к типу int.  
Также было определено, что колонки Geography, Gender являются категориальными - это значит, что тип данных object можно привести к int с помощью ассоциаций.

#### Исправление датасета 
Сначала обработаем регистры в столбцах и в значениях столбцов

In [ ]:
# Исправление регистров столбцов и приведение к "змеиному регистру"
data_set.columns = (
    data_set.columns
    .str.replace(r'(?<=[a-z])([A-Z])', r'_\1', regex=True)
    .str.lower()
)
data_set.columns.to_list()

Далее удалим столбцы row_number и customer_id, но столбец surname оставим для дальнейшего анализа

In [ ]:
data_set = data_set.drop(["row_number", "customer_id"], axis=1)

При анализе столбца фамилий было обнаружено, что из 10 000 фамилий только уникальных фамилий только 2932, а значит для оптимизации памяти и сохранения конфиденциальности клиентов можно создать ассоциативный словарь фамилий

In [ ]:
# приведение имен к нижнему регистру и удаление спецсимволы
data_set["surname"] = data_set["surname"].str.lower().str.replace(r'[^\w_]', '', regex=True)

# в pandas есть функция factorize, которая создает список уникальных значений, а в колонке заменяет фамилии на число
data_set["surname"], surnames = pd.factorize(data_set["surname"])

# т.к factorize возвращает только список уникальных фамилий, необходимо сделать 2 (на всякий случай) словаря форматов: число:фамилия и фамилия:число
SURNAMES_VAL = dict(enumerate(surnames))
SURNAMES_ID = {val: key for key, val in SURNAMES_VAL.items()}

# для самостоятельной проверки корректности преобразования вывод списка имен
SURNAMES_VAL

Для столбцов gender и geography регистр менять не будем, а заменим строковые (object) значения на числовые, где 1 это будет male, а 2 - female для gender и 1 - France, 2 - Spain, 3 - Germany для geography

In [ ]:
# создание ассоциаций
GENDER_VAL = {
    1: "male",
    2: "female"
}

GEOGRAPHY_VAL = {
    1: "france",
    2: "spain",
    3: "germany"
}
# обратный словарь на всякий случай
GENDER_ID = {val: key for key, val in GENDER_VAL.items()}
GEOGRAPHY_ID = {val: key for key, val in GEOGRAPHY_VAL.items()}

# функции для отображения значений (возможно не понадобится вообще)
def get_gender_value(num: int) -> str:
    return GENDER_VAL[num]

def get_gender_id(val: str) -> int:
    return GENDER_ID[val]

def get_geography_value(num: int) -> str:
    return GEOGRAPHY_VAL[num]

def get_geography_id(val: str) -> int:
    return GEOGRAPHY_ID[val]


In [ ]:
# замена значений на ассоциации
data_set["gender"] = data_set["gender"].map({"Male": 1, "Female": 2})
data_set["geography"] = data_set["geography"].map({"France": 1, "Spain": 2, "Germany": 3})
data_set.sample(10)

Обратим внимание, что в колонке geography неверно установился тип данных  
Возможно это связано с наличием пропусков в колонке. Далее будет рассмотрена обработка пропусков и преобразование типов данных

#### Обработка пропусков
В датасете было обнаружено 4 пропуска:

In [ ]:
rows_with_missing = data_set[data_set.isna().any(axis=1)]
rows_with_missing

Для каждой замены необходимо выбрать те методы, которые сохранят статические свойства данных:
1. Для категориального значения в столбце geography воспользуемся модой (превалирующие значения) для сохранения распределения
2. Для значения в столбце age воспользуемся медианой, которая устойчива к выбросам (среднее не учитывает возможные выбросы)
3. Для значения в столбце has_cr_card воспользуемся логикой)) Если данные о кредитной карте отсутствуют в системе, то логично предположить, что у клиента нет кредитной карты
4. Для значения в столбце is_active_member для минимизации искажения воспользуемся медианой

In [ ]:
# geography 
data_set.loc[6, 'geography'] = data_set['geography'].mode()[0]

# age
data_set.loc[9, 'age'] = data_set['age'].median()

# has_cr_card
data_set.loc[4, 'has_cr_card'] = 0

# is_active_member
data_set.loc[8, 'is_active_member'] = data_set['is_active_member'].median()

# перепроверка пропусков
data_set.isnull().sum()

#### Приведение значений столбцов к необходимым типам данных
В ходе анализа датасета установлено, что необходимо изменить следующие типы данных:
1. credit_score с int64 на int32 (оптимизация памяти)
2. geography с float на int8 (изначально неверный тип данных)
3. gender с int64 на int8 (оптимизация памяти)
4. age с float на int8 (изначально неверный тип данных)
5. tenure c int64 на int8 (оптимизация памяти)
6. num_of_products с int32 на int16 (оптимизация памяти)
7. has_cr_card с float на bool (неверный тип данных)
8. is_active_member c float на bool (неверный тип данных)
9.  exited с int на bool (неверный тип данных)
    
В основном изменения типов данных нужны для оптимизации памяти, но присутствуют и неверные типы данных

Также нужно учесть особенности работы pandas с работой чисел с плавающей точкой (float):  
float64 является стандартным представлением чисел с плавающей точкой в pandas и в начальных настройках отображения данных автоматически показывает только 2 знака после точки, хотя float64 имеет точность ~16 знаков после запятой. Изменение данного типа данных на float32 может привести к большему накоплению ошибок, чем float64.  
Для денежных форматов необходимо использовать тип данных Decimal, но т.к датасет носит учебный характер - можно оставить float64.

In [ ]:
data_set = data_set.astype({
    'credit_score': 'int32',
    'geography': 'int8',
    'gender': 'int8',
    'age': 'int8',
    'tenure': 'int8',
    'num_of_products': 'int16',
    'has_cr_card': 'bool',
    'is_active_member': 'bool',
    'exited': 'bool'
})

# проверка отработало ли преобразование типов корректно или нет
data_set.dtypes

#### Поиск дубликатов

In [ ]:
# Поиск дубликатов
data_set.duplicated().sum()

In [ ]:
# т.к нашлись дубликаты, выведем их
duplicates = data_set[data_set.duplicated(keep=False)]
duplicates

На печати видно, что дубликаты полностью идентичны (9998 с 9999 и 10000 с 10001).
Это может быть связано с разными причинами, например, ошибка при выгрузке датасета (на api или бд могли произойти сбои, которые вызвали повторную запись - это маловероятно, но не равно 0). Также стоит учесть, что это датасет для обучающихся и дублирование данных могло произойти нарочно.  
Также стоит учесть, что могло произойти некорректное объединение нескольких датасетов в один.  
Для устранения дублирования удалим дубликаты с сохранением одного экземпляра.

In [ ]:
data_set = data_set.drop_duplicates()
data_set.duplicated().sum()

#### Проверка датасета после предобработки данных

In [ ]:
data_set.head()

In [ ]:
data_set.info()

In [ ]:
data_set.memory_usage(deep=True)

#### Вывод по первому этапу

В ходе предварительного анализа и обработки набора данных были выполнены следующие шаги и устранены выявленные проблемы:

1. Были просмотрены первые и последние строки, случайная выборка, структура датасета, типы данных и использование памяти.  
   Это позволило сформировать общее представление о содержимом датасета и выявить потенциальные проблемы: наличие ненужных столбцов, неунифицированные имена, категориальные признаки в строковом формате, пропуски в данных, дублирование данных и несоответствие типов данных.

2. При анализе датасета были выявлены:
   - Наличие неинформативных столбцов (`row_number`, `customer_id`), которые были удалены из датасета;
   - Несоблюдение стиля наименования столбцов (`"змеиного регистра"`). В процессе обработки все строковые значения (включая название столбцов) были приведены к этому стилю;
   - Наличие строкового формата у категориальных признаков (`gender`, `geography`). Для решения данной проблемы были созданы отдельные словари, а в датасете использованы цифровые ключи;
   - Наличие пустых значений и дубликатов, которые в процессе были устранены;
   - Несоответствие типов данных. После предобработки все данные были приведены к необходимым типам

3. Столбец `surname` был очищен от специальных символов и приведён к нижнему регистру, после чего был закодирован с помощью pd.factorize, что позволило превратить строковый столбец в числовой идентификатор.
   
Таким образом данные были подготовлены к дальнейшему анализу.
